In [6]:
!pip install beautifulsoup4

  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.14.2-py3-none-any.whl (106 kB)
Using cached soupsieve-2.8-py3-none-any.whl (36 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [beautifulsoup4]m [beautifulsoup4]


In [8]:
!pip install webdriver_manager

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp314-cp314-macosx_10_13_universal2.whl.metadata (37 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.4-cp314-cp314-macosx_10_13_universal2.whl (207 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [webdriver_manager]requests]


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import logging

# Set up logging
logging.basicConfig(filename='scraper_errors.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Create base output directory
output_dir = "fbref_premier_league_data_insider01"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# ADDED TO AUTOMATE DRIVER INSTALLATION AND SETUP
# TEST IF IT WORKS FINE ON YOUR MACHINE
# IF NOT PLEASE REVERT TO MANUAL SETUP AS COMMENTED BELOW
# IF THIS WORKS, MAKE SURE TO INSTALL THE 'webdriver-manager' PACKAGE VIA PIP
# pip install webdriver-manager
# OR JUST RUN PIP INSTALL REQUREMENTS.TXT

from webdriver_manager.chrome import ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


# Initialize WebDriver (headless mode disabled for debugging)
#ser = Service(r"C:\Users\vanim\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
#driver = webdriver.Chrome(service=ser)

# Define seasons and teams (filtered for 2015-2016 Premier League teams)
START_YEAR = 2015
END_YEAR = 2024
# Full player list for later use
# name= ['Pierre-Emerick-Aubameyang', 'David-Luiz', 'Bernd-Leno', 'Granit-Xhaka', 'Nicolas-Pépé', 'Alexandre-Lacazette', 
#        'Bukayo-Saka', 'Mattéo-Guendouzi', 'Sead-Kolašinac', 'Sokratis-Papastathopoulos', 'Dani-Ceballos', 'Mesut-Özil', 
#        'Lucas-Torreira', 'Ainsley-Maitland-Niles', 'Shkodran-Mustafi', 'Héctor-Bellerín', 'Calum-Chambers', 'Kieran-Tierney',
#        'Joe-Willock', 'Emiliano-Martínez','Eddie-Nketiah', 'Reiss-Nelson', 'Gabriel-Martinelli', 'Rob-Holding', 'Nacho-Monreal', 'Cédric-Soares',
#        'Pablo-Marí', 'Henrikh-Mkhitaryan', 'Emile-Smith-Rowe', 'Tyreece-John-Jules', 'Matt-Macey', 'Konstantinos-Mavropanos', 'Zech-Medley',
#        'Matthew-Smith','Idrissa-Gana-Gueye', 'Ashley-Westwood', 'Joleon-Lescott', 'Brad-Guzan', 'Leandro-Bacuna', 'Jordan-Ayew', 'Alan-Hutton','Micah-Richards', 
#        'Jordan-Veretout', 'Scott-Sinclair', 'Aly-Cissokho', 'Carles-Gil', 'Ciaran-Clark', 'Carlos-Sánchez', 'Rudy-Gestede', 'Gabriel-Agbonlahor', 'Jores-Okore', 'Mark-Bunn',
#        'Jordan-Amavi', 'Jack-Grealish', 'Kieran-Richardson', 'Kevin-Toner', 'Libor-Kozák', 'Jordan-Lyden', 'José-Ángel-Crespo', 'Adama-Traoré', 'André-Green', "Charles-N'Zogbia", 
#        'Rushian-Hepburn-Murphy', 'Nathan-Baker', 'Joe-Bennett', 'Joe-Cole', 'Keinan-Davis', 'Tiago-Ilori', 'Niall-Mason''Andrew-Surman', 'Simon-Francis', 'Charlie-Daniels', 
#        'Steve-Cook', 'Matt-Ritchie', 'Artur-Boruc', 'Dan-Gosling', 'Joshua-King', 'Adam-Smith', 'Harry-Arter', 
#        'Junior-Stanislas', 'Marc-Pugh', 'Benik-Afobe', 'Tommy-Elphick', 'Max-Gradel', 'Sylvain-Distin', 'Callum-Wilson', 
#        "Eunan-O'Kane", 'Glenn-Murray', 'Adam-Federici', 'Lewis-Grabban', 'Lee-Tomlin', 'Yann-Kermorgant', 'Juan-Iturbe', 'Ryan-Allsop',
#        'Tokelo-Rantie', 'Shaun-MacDonald', 'Tyrone-Mings', 'Joe-Bennett', 'Matt-Butcher', 'Baily-Cargill', 'Jordan-Holmes', 
#        'Corey-Jordan', 'Rhoys-Wiggins','César-Azpilicueta', 'Branislav-Ivanović', 'Cesc-Fàbregas', 'Willian', 'Nemanja-Matić', 
#        'Diego-Costa', 'Eden-Hazard', 'Pedro', 'John-Terry', 'Thibaut-Courtois', 'Gary-Cahill', 'Kurt-Zouma', 'Oscar', 
#        'John-Obi-Mikel', 'Asmir-Begović', 'Baba-Rahman', 'Ramires', 'Kenedy', 'Ruben-Loftus-Cheek', 'Bertrand-Traoré', 'Loïc-Rémy', 
#        'Matt-Miazga', 'Radamel-Falcao', 'Alexandre-Pato', 'Tammy-Abraham', 'Fikayo-Tomori', 'Juan-Cuadrado', 'Jake-Clarke-Salter',
#        'Ola-Aina', 'Marco-Amelia', 'Jamal-Blackman', 'Charlie-Colkett', 'Papy-Djilobodji', 'Victor-Moses', 'Kasey-Palmer','Scott-Dann',
#        'Pape-Souaré', 'Damien-Delaney', 'Yohan-Cabaye', 'Jason-Puncheon', 'Joel-Ward', 'Wilfried-Zaha', 'Wayne-Hennessey', 'James-McArthur', 
#        'Yannick-Bolasie', 'Mile-Jedinak', 'Connor-Wickham', 'Bakary-Sako', 'Martin-Kelly', 'Joe-Ledley', 'Dwight-Gayle', 'Jordon-Mutch',
#        'Brede-Hangeland', 'Alex-McCarthy', 'Emmanuel-Adebayor', 'Lee-Chung-yong', 'Fraizer-Campbell', 'Adrian-Mariappa', 'Julián-Speroni', 
#        'Glenn-Murray', 'Marouane-Chamakh', 'Patrick-Bamford', 'Sullay-Kaikai', 'Hiram-Boateng', 'Jonny-Williams', 'Luke-Croll', 'Luke-Dreher',
#        'Connor-Dymond', 'Jake-Gray','Romelu-Lukaku', 'Ross-Barkley', 'Gareth-Barry', 'John-Stones', 'James-McCarthy', 'Séamus-Coleman', 'Tim-Howard', 
#        'Ramiro-Funes-Mori', 'Phil-Jagielka', 'Tom-Cleverley', 'Aaron-Lennon', 'Leighton-Baines', 'Arouna-Koné', 'Gerard-Deulofeu', 
#        'Brendan-Galloway', 'Joel-Robles', 'Bryan-Oviedo', 'Kevin-Mirallas', 'Muhamed-Bešić', 'Steven-Naismith', 'Matthew-Pennington', 
#        'Tyias-Browning', 'Darron-Gibson', 'Leon-Osman', 'Oumar-Niasse', 'Tom-Davies', 'Kieran-Dowell', 'Steven-Pienaar', 'Jonjoe-Kenny', 
#        'Tony-Hibbert', 'Callum-Connolly', 'Mason-Holgate', 'Conor-McAleny', 'Aiden-McGeady', 'Jindřich-Staněk','Wes-Morgan', 'Kasper-Schmeichel',
#        'Jamie-Vardy', 'Riyad-Mahrez', 'Robert-Huth', 'Danny-Drinkwater', 'Marc-Albrighton', "N'Golo-Kanté", 'Christian-Fuchs',
#        'Danny-Simpson', 'Shinji-Okazaki', 'Jeffrey-Schlupp', 'Andy-King', 'Leonardo-Ulloa', 'Ritchie-De-Laet', 'Marcin-Wasilewski', 
#        'Gökhan-Inler', 'Demarai-Gray', 'Daniel-Amartey', 'Nathan-Dyer', 'Yohan-Benalouane', 'Andrej-Kramarić', 'Joe-Dodoo',
#        'Ben-Chilwell', 'Dean-Hammond', 'Jonny-Maddison', 'Mark-Schwarzer', 'Simon-Mignolet', 'Nathaniel-Clyne', 'Alberto-Moreno', 'Emre-Can', 'James-Milner',
#        'Philippe-Coutinho', 'Roberto-Firmino', 'Adam-Lallana', 'Dejan-Lovren', 'Lucas-Leiva', 'Martin-Škrtel', 'Mamadou-Sakho', 
#        'Jordan-Henderson', 'Christian-Benteke', 'Jordon-Ibe', 'Daniel-Sturridge', 'Kolo-Touré', 'Joe-Allen', 'Divock-Origi', 'Kevin-Stewart',
#        'Joe-Gomez', 'Jon-Flanagan', 'Sheyi-Ojo', 'Danny-Ings', 'Brad-Smith', 'Ádám-Bogdán', 'Danny-Ward', 'Connor-Randall',
#        'Cameron-Brannagan', 'Pedro-Chirivella', 'Jordan-Rossiter', 'Sergi-Canós', 'Steven-Caulker', 'João-Carlos-Teixeira',
#        'Ryan-Fulton', 'Tiago-Ilori', 'Jerome-Sinclair', 'Joe-Hart', 'Fernandinho', 'Nicolás-Otamendi', 'Sergio-Agüero', 
#        'Yaya-Touré', 'Bacary-Sagna', 'Aleksandar-Kolarov', 'Jesús-Navas', 'Eliaquim-Mangala', 'Raheem-Sterling', 'Kevin-De-Bruyne',
#        'David-Silva', 'Fernando', 'Wilfried-Bony', 'Vincent-Kompany', 'Gaël-Clichy', 'Pablo-Zabaleta', 'Martín-Demichelis',
#        'Fabian-Delph', 'Kelechi-Iheanacho','Samir-Nasri', 'Willy-Caballero', 'Bersant-Celina', 'Manu-García', 'Patrick-Roberts', 
#        'Angeliño', 'Brandon-Barker', 'Jason-Denayer', 'George-Evans', 'Aleix-García', 'James-Horsfield', 'Cameron-Humphreys', 
#        'Pablo-Maffeo', 'Richard-Wright', 'Chris-Smalling', 'Daley-Blind', 'David-de-Gea', 'Juan-Mata', 'Anthony-Martial', 'Wayne-Rooney', 
#        'Morgan-Schneiderlin', 'Matteo-Darmian', 'Michael-Carrick', 'Jesse-Lingard', 'Ander-Herrera', 'Memphis', 'Marcos-Rojo',
#        'Bastian-Schweinsteiger', 'Marouane-Fellaini', 'Ashley-Young', 'Marcus-Rashford', 'Antonio-Valencia', 'Cameron-Borthwick-Jackson', 
#        'Phil-Jones', 'Luke-Shaw', 'Sergio-Romero', 'Guillermo-Varela', 'Paddy-McNair', 'Timothy-Fosu-Mensah', 'Adnan-Januzaj',
#        'Andreas-Pereira', 'Donald-Love', 'Javier-Hernández', 'Nick-Powell', 'James-Wilson', 'Will-Keane', 'James-Weir', 'Sean-Goss', 'Sam-Johnstone',
#        'Joe-Riley', 'Joe-Rothwell', 'Axel-Tuanzebe', 'Moussa-Sissoko', 'Georginio-Wijnaldum', 'Chancel-Mbemba', 'Daryl-Janmaat', 'Jack-Colback', 'Fabricio-Coloccini',
#        'Vurnon-Anita', 'Paul-Dummett', 'Aleksandar-Mitrović', 'Ayoze-Pérez', 'Rob-Elliot', 'Cheick-Tioté', 'Papiss-Cissé', 
#        'Andros-Townsend', 'Jonjo-Shelvey', 'Jamaal-Lascelles', 'Karl-Darlow', 'Steven-Taylor', 'Tim-Krul', 'Massadio-Haïdara', 
#        'Siem-de-Jong', 'Florian-Thauvin', 'Rolando-Aarons', 'Gabriel-Obertan', 'Yoan-Gouffran', 'Henri-Saivet', 'Kevin-Mbabu', 
#        'Emmanuel-Rivière', 'Seydou-Doumbia', 'Ivan-Toney', 'Jamie-Sterry', 'Daniel-Barlaser', 'Liam-Gibson', 'Sylvain-Marveaux', 'Callum-Williams',
#        'Mike-Williamson', 'Freddie-Woodman', 'Robbie-Brady', 'Jonny-Howson', 'Sébastien-Bassong', 'Russell-Martin', 
#        'John-Ruddy', 'Wes-Hoolahan', 'Nathan-Redmond', 'Alexander-Tettey', 'Ryan-Bennett', 'Martin-Olsson', 'Cameron-Jerome',
#        "Gary-O'Neil", 'Dieumerci-Mbokani', 'Graham-Dorrans', 'Matthew-Jarvis', 'Declan-Rudd', 'Steven-Naismith', 'Timm-Klose', 'Andre-Wisdom', 'Ivo-Pinto', 
#        'Steven-Whittaker', 'Youssouf-Mulumbu', 'Vadis-Odjidja-Ofoe', 'Lewis-Grabban', 'Patrick-Bamford', 'Bradley-Johnson',
#        'Kyle-Lafferty', 'Gary-Hooper', 'Tony-Andreu', 'Ricky-van-Wolfswinkel','José-Fonte', 'Virgil-van-Dijk', 'Ryan-Bertrand', 'Steven-Davis', 'Sadio-Mané', 'Victor-Wanyama', 
#        'Dušan-Tadić', 'Graziano-Pellè', 'Shane-Long', 'Cédric-Soares', 'Jordy-Clasie', 'Fraser-Forster', 'Oriol-Romeu', 'Maarten-Stekelenburg', 'James-Ward-Prowse', 
#        'Matt-Targett', 'Cuco-Martina', 'Maya-Yoshida', 'Jay-Rodriguez', 'Charlie-Austin', 'Paulo-Gazzaniga', 'Steven-Caulker', 'Kelvin-Davis', 'Juanmi',
#        'Gastón-Ramírez', 'Harrison-Reed', 'Olufela-Olomola', 'Ryan-Seager','Glenn-Whelan', 'Erik-Pieters', 'Marko-Arnautović',
#        'Jack-Butland', 'Philipp-Wollscheid', 'Geoff-Cameron', 'Xherdan-Shaqiri', 'Glen-Johnson', 'Ibrahim-Afellay', 'Bojan-Krkić', 'Ryan-Shawcross',
#        'Jonathan-Walters', 'Giannelli-Imbula', 'Mame-Biram-Diouf', 'Marc-Muniesa', 'Charlie-Adam', 'Joselu', 'Phil-Bardsley', 'Marco-van-Ginkel', 'Peter-Crouch',
#        'Jakob-Haugaard', 'Shay-Given', 'Marc-Wilson', 'Stephen-Ireland', 'Peter-Odemwingie', 'Dionatan-Teixeira', 
#        'Steve-Sidwell', 'Daniel-Bachmann', 'Moha-El-Ouriachi', 'Ollie-Shenton',"Yann-M'Vila", 'Patrick-van-Aanholt', 'Jermain-Defoe', 'Lee-Cattermole', 'Billy-Jones', 
#        "John-O'Shea", 'Younès-Kaboul', 'Fabio-Borini', 'DeAndre-Yedlin', 'Vito-Mannone', 'Costel-Pantilimon', 'Lamine-Koné', 'Jeremain-Lens', 'Sebastián-Coates',
#        'Jan-Kirchhoff', 'Wahbi-Khazri', 'Adam-Johnson', 'Steven-Fletcher', 'Jack-Rodwell', 'Ola-Toivonen',
#        'Duncan-Watmore', 'Sebastian-Larsson', 'Wes-Brown', "Dame-N'Doye", 'Jordi-Gómez', 'Danny-Graham', 'Jordan-Pickford',
#        'Thomas-Robson', 'Rees-Greenwood', 'George-Honeyman', 'Adam-Matthews', 'Tom-Beadling', 'Liam-Bridcutt', 'Emanuele-Giaccherini',
#        'Lynden-Gooch', 'Mikael-Mandron','Łukasz-Fabiański', 'Ashley-Williams', 'André-Ayew', 'Neil-Taylor', 'Gylfi-Sigurðsson',
#        'Federico-Fernández', 'Jack-Cork', 'Wayne-Routledge', 'Ki-Sung-yueng', 'Àngel-Rangel', 'Kyle-Naughton', 'Leon-Britton', 
#        'Bafétimbi-Gomis', 'Jefferson-Montero', 'Jonjo-Shelvey', 'Leroy-Fer', 'Alberto-Paloschi', 'Modou-Barrow', 'Jordi-Amat', 'Stephen-Kingsley', 
#        'Kyle-Bartley', 'Éder', 'Matt-Grimes', 'Kristoffer-Nordfeldt', 'Marvin-Emnes', 'Jay-Fulton', 'Nathan-Dyer', 'Kenji-Gorré', 
#        'Franck-Tabanou', 'Gerhard-Tremmel', 'Josh-Vickers', 'Toby-Alderweireld', 'Harry-Kane', 'Hugo-Lloris', 'Eric-Dier', 'Kyle-Walker', 'Christian-Eriksen', 
#        'Jan-Vertonghen', 'Dele-Alli', 'Érik-Lamela', 'Mousa-Dembélé', 'Danny-Rose', 'Ben-Davies', 'Son-Heung-min', 'Nacer-Chadli', 'Kevin-Wimmer', 'Ryan-Mason', 'Kieran-Trippier',
#        'Tom-Carroll', 'Nabil-Bentaleb', 'Michel-Vorm', "Clinton-N'Jie", 'Josh-Onomah', 'Andros-Townsend', 'Alex-Pritchard', 'Federico-Fazio',
#        'Kyle-Walker-Peters', 'Harry-Winks','Heurelho-Gomes', 'Troy-Deeney', 'Odion-Ighalo', 'Craig-Cathcart', 'Étienne-Capoue', 'Ben-Watson', 
#        'Allan-Nyom', 'José-Manuel-Jurado', 'Almen-Abdi', 'Miguel-Britos', 'Nathan-Aké', 'Sebastian-Prödl', 'Ikechi-Anya', 'Valon-Behrami', 
#        'José-Holebas', 'Mario-Suárez', 'Juan-Carlos-Paredes', 'Nordin-Amrabat', 'Adlène-Guedioura', 'Miguel-Layún', 
#        'Steven-Berghuis', 'Víctor-Ibarbo', 'Alessandro-Diamanti', 'Giedrius-Arlauskis', 'Obbi-Oulare', 'Gabriele-Angella',
#        'Lloyd-Dyer', 'Fernando-Forestieri', 'Rene-Gilmartin', 'Tommie-Hoban', 'Costel-Pantilimon', 'Daniel-Pudil', 'Matěj-Vydra', 
#        'Craig-Dawson', 'Darren-Fletcher', 'Gareth-McAuley', 'Claudio-Yacob', 'Jonny-Evans', 'Salomón-Rondón', 'James-McClean',
#        'Jonas-Olsson', 'Boaz-Myhill', 'Stéphane-Sessègnon', 'Craig-Gardner', 'Chris-Brunt', 'Saido-Berahino', 'James-Morrison',
#        'Ben-Foster', 'James-Chester', 'Rickie-Lambert', 'Sandro', 'Victor-Anichebe', 'Jonathan-Leko',
#        'Callum-McManaman', 'Joleon-Lescott', 'Alex-Pritchard', 'Sébastien-Pocognoli', 'Cristian-Gamboa', 
#        'Tyler-Roberts', 'Serge-Gnabry', 'Sam-Field', 'Brown-Ideye', 'Anders-Lindegaard', 'Alex-Palmer', 'Jack-Rose', 
#        'Aaron-Cresswell', 'Mark-Noble', 'Cheikhou-Kouyaté', 'Adrián', 'Dimitri-Payet', 'Angelo-Ogbonna', 'Winston-Reid',
#        'Michail-Antonio', 'James-Tomkins', 'Manuel-Lanzini', 'Diafra-Sakho', 'James-Collins', 'Andy-Carroll', 
#        'Carl-Jenkinson', 'Victor-Moses', 'Pedro-Obiang', 'Enner-Valencia', 'Mauro-Zárate', 'Alexandre-Song', 
#        'Darren-Randolph', 'Emmanuel-Emenike', 'Reece-Oxford', 'Sam-Byram', 'Nikica-Jelavić', 'Kevin-Nolan', 
#        'Matthew-Jarvis', 'Modibo-Maïga', 'Josh-Cullen', 'Marcus-Browne', 'George-Dobson', 'Stephen-Hendrie',
#        'Doneil-Henry', 'Kyle-Knoyle', 'Elliot-Lee', "Joey-O'Brien", 'Djair-Parfitt-Williams', 'Diego-Poyet', 
#        'Martin-Samuelsen', 'Raphael-Spiegel', ]
# Full team list for later seasons
teams_full = [
    "Liverpool", "Arsenal", "Manchester-City", "Chelsea", "Newcastle-Utd",
    "Aston-Villa", "Nott'ham-Forest", "Brighton", "Bournemouth", "Brentford",
    "Fulham", "Crystal-Palace", "Everton", "West-Ham", "Manchester-Utd",
    "Wolves", "Tottenham", "Leicester-City", "Ipswich-Town", "Southampton","Middlesbrough","Hull-City","Burnley","Swansea-City","Stoke-City","West-Bromwich-Albion",
    "Huddersfield-Town","Norwich-City","Sheffield-United","Cardiff-City","Leeds-United","Luton-Town","Watford","Sunderland","Leicester"]

# Squad IDs
squad_ids = {
    "Liverpool": "822bd0ba",
    "Arsenal": "18bb7c10",
    "Manchester-City": "b8fd03ef",
    "Chelsea": "cff3d9bb",
    "Newcastle-Utd": "b2b47a98",
    "Aston-Villa": "8602292d",
    "Nott'ham-Forest": "e4a775cb",
    "Brighton": "d07537b9",
    "Bournemouth": "4ba7cbea",
    "Brentford": "cd051869",
    "Fulham": "fd962109",
    "Crystal-Palace": "47c64c55",
    "Everton": "d3fd31cc",
    "West-Ham": "7c21e445",
    "Manchester-Utd": "19538871",
    "Wolves": "8cec06e1",
    "Tottenham": "361ca564",
    "Leicester-City": "a2d435b3",
    "Ipswich-Town": "b74092de",
    "Southampton": "33c895d4",
    "Norwich-City": "1c781004",
    "Stoke-City": "17892952",
    "Swansea-City": "fb10988f",
    "Watford": "2abfe087",
    
    "Sunderland": "8ef52968",
    "Leicester": "a2fb4471",
    "Middlesbrough":"7f59c601",
    "Hull-City":"bd8769d1",
    "Burnley":"943e8050",
    "West-Bromwich-Albion":"60c6b05f",
    "Huddersfield-Town":"f5922ca5",
    "Sheffield-United":"1df6b87e",
    "Cardiff-City":"75fae011",
    "Leeds-United":"5bfb9659",
    
    "Luton-Town":"e297cd13",

}

def scrape_team(driver, url, team, season, season_dir, max_retries=2):
    for attempt in range(max_retries):
        try:
            print(f"Scraping (Attempt {attempt + 1}): {url}")
            driver.get(url)
            
            # Wait for any table with 'stats_standard' in the ID
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.XPATH, "//table[contains(@id, 'stats_standard')]"))
            )
            
            # Parse page source
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            
            # Find the table (try multiple possible IDs)
            table = None
            for table_id in ["stats_standard_9", "stats_standard", "stats_standard_12"]:
                table = soup.find("table", {"id": table_id})
                if table:
                    break
            
            if table is None:
                logging.error(f"Table not found for {team} in {season}")
                print(f"Table not found for {team} in {season}. Skipping...")
                return False
            
            # Convert to DataFrame and save
            df = pd.read_html(str(table))[0]
            
            
            # filename = os.path.join(season_dir, f"{team}_PremierLeague.csv")
            # df.to_csv(filename, index=False)
            # print(f"Saved: {filename}")
            # return True
            
        except Exception as e:
            logging.error(f"Error scraping {team} for {season} (Attempt {attempt + 1}): {str(e)}")
            print(f"Error scraping {team} for {season} (Attempt {attempt + 1}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait before retrying
            continue
    return False

try:
    for year in range(START_YEAR, END_YEAR + 1):
        next_year = year + 1
        season = f"{year}-{next_year}"
        
        # Create season subdirectory
        season_dir = os.path.join(output_dir, season)
        if not os.path.exists(season_dir):
            os.makedirs(season_dir)
            print(f"Created directory: {season_dir}")
        
 
        
        for team in teams_full:
            squad_id = squad_ids.get(team)
            if not squad_id:
                print(f"No squad ID for {team}. Skipping...")
                logging.error(f"No squad ID for {team} in {season}")
                continue
                
            url = f"https://fbref.com/en/squads/{squad_id}/{season}/{team}-Stats"
            scrape_team(driver, url, team, season, season_dir)
            time.sleep(2)  # Delay to avoid rate limiting

finally:
    print("Closing browser...")
    time.sleep(2)
    driver.quit()

Scraping (Attempt 1): https://fbref.com/en/squads/822bd0ba/2015-2016/Liverpool-Stats


/var/folders/q8/rfr14j2j62n1zftp1z06v_bm0000gn/T/ipykernel_67041/142154947.py:209: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Error scraping Liverpool for 2015-2016 (Attempt 1): Missing optional dependency 'lxml'.  Use pip or conda to install lxml.
Scraping (Attempt 2): https://fbref.com/en/squads/822bd0ba/2015-2016/Liverpool-Stats
Error scraping Liverpool for 2015-2016 (Attempt 2): HTTPConnectionPool(host='localhost', port=53024): Read timed out. (read timeout=120)
Scraping (Attempt 1): https://fbref.com/en/squads/18bb7c10/2015-2016/Arsenal-Stats
Error scraping Arsenal for 2015-2016 (Attempt 1): HTTPConnectionPool(host='localhost', port=53024): Read timed out. (read timeout=120)
Scraping (Attempt 2): https://fbref.com/en/squads/18bb7c10/2015-2016/Arsenal-Stats
Error scraping Arsenal for 2015-2016 (Attempt 2): HTTPConnectionPool(host='localhost', port=53024): Read timed out. (read timeout=120)
Scraping (Attempt 1): https://fbref.com/en/squads/b8fd03ef/2015-2016/Manchester-City-Stats
Error scraping Manchester-City for 2015-2016 (Attempt 1): HTTPConnectionPool(host='localhost', port=53024): Read timed out. (rea